In [1]:
import os
import keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Conv2D, Dropout, Flatten, Dense, Activation, MaxPool2D

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# training data augmentation
train_datagen =image.ImageDataGenerator(
   rescale =1/255, horizontal_flip =True ,zoom_range =0.2, shear_range =0.2) 
train_data =train_datagen.flow_from_directory(directory="/content/drive/MyDrive/Data/Train",target_size= (64,64), 
                                              batch_size= 16,class_mode = 'categorical')

Found 2425 images belonging to 2 classes.


In [ ]:
test_data.class_indices

In [4]:
# test data augmentation
test_datagen =image.ImageDataGenerator(
   rescale =1/255)
test_data =test_datagen.flow_from_directory(directory="/content/drive/MyDrive/Data/Test",target_size= (64,64),
                                             batch_size = 16,
                                             class_mode = 'categorical')

Found 871 images belonging to 2 classes.


In [ ]:
test_data.class_indices

In [ ]:
# Define a function to plot images and their labels
def plot_images(images, labels, title):
    fig, axs = plt.subplots(nrows=1, ncols=len(images), figsize=(15,5))
    fig.suptitle(title, fontsize=16)

    for i in range(len(images)):
        axs[i].imshow(images[i])
        axs[i].set_title('Label: ' + str(labels[i]))
        axs[i].axis('off')

    plt.show()

# Select 5 random images from the Train dataset
train_image_paths = np.random.choice(train_data.filepaths, size=5, replace=False)
train_images = [image.load_img(path, target_size=(256,256)) for path in train_image_paths]
train_labels = [os.path.basename(os.path.dirname(path)) for path in train_image_paths]

# Select 5 random images from the Test dataset
test_image_paths = np.random.choice(test_data.filepaths, size=5, replace=False)
test_images = [image.load_img(path, target_size=(256,256)) for path in test_image_paths]
test_labels = [os.path.basename(os.path.dirname(path)) for path in test_image_paths]

# Convert images to arrays
train_images = np.array([image.img_to_array(img) for img in train_images])
test_images = np.array([image.img_to_array(img) for img in test_images])

# Rescale pixel values to range [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Plot the selected images with their labels
plot_images(train_images, train_labels, 'Randomly selected images from the Train dataset')
plot_images(test_images, test_labels, 'Randomly selected images from the Test dataset')

In [23]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape = (64,64,3), use_bias=True))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape = (64,64,3), use_bias=True))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', use_bias=True))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))

model.add(Flatten())
model.add(Dense(256, activation= 'relu', use_bias=True))
model.add(Dropout(rate=0.2))

model.add(Dense(32, activation= 'relu', use_bias=True))

model.add(Dense(2, activation='softmax', use_bias=True))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 31, 31, 32)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 14, 14, 32)       

In [15]:
cnnModel = model.fit(train_data, steps_per_epoch=150, epochs=17, validation_data=test_data)

Epoch 1/17
150/150 [==============================] - 63s 415ms/step - loss: 0.1684 - accuracy: 0.9398 - val_loss: 0.6424 - val_accuracy: 0.6498
Epoch 2/17
150/150 [==============================] - 62s 416ms/step - loss: 0.1367 - accuracy: 0.9419 - val_loss: 0.5577 - val_accuracy: 0.7440
Epoch 3/17
150/150 [==============================] - 65s 433ms/step - loss: 0.1742 - accuracy: 0.9344 - val_loss: 0.3095 - val_accuracy: 0.8726
Epoch 4/17
150/150 [==============================] - 65s 431ms/step - loss: 0.1616 - accuracy: 0.9373 - val_loss: 0.5124 - val_accuracy: 0.7532
Epoch 5/17
150/150 [==============================] - 61s 407ms/step - loss: 0.1513 - accuracy: 0.9386 - val_loss: 0.3309 - val_accuracy: 0.8898
Epoch 6/17
150/150 [==============================] - 65s 432ms/step - loss: 0.1522 - accuracy: 0.9407 - val_loss: 0.2337 - val_accuracy: 0.9116
Epoch 7/17
150/150 [==============================] - 64s 428ms/step - loss: 0.1296 - accuracy: 0.9486 - val_loss: 0.3018 - val_ac

In [22]:
path ="/content/drive/MyDrive/Data/Test/Fire/fire (126).jpg"

img =image.load_img(path,target_size=(64,64))
img =image.img_to_array(img)/255
img =np.array([img])
img.shape
prediction = model.predict(img)
if prediction.any() > 0.5:
    print('Fire detected')
else:
    print('No Fire detected.')

1/1 [==============================] - 0s 20ms/step
Fire detected


In [25]:
path ="/content/drive/MyDrive/new/Fire/fire7.jpg"

img =image.load_img(path,target_size=(64,64))
img =image.img_to_array(img)/255
img =np.array([img])
img.shape
prediction = model.predict(img)
if prediction.any() > 0.5:
    print('Fire detected')
else:
    print('No Fire detected.')

1/1 [==============================] - 0s 74ms/step
Fire detected


In [ ]:
# Get information about the trainable parameters
num_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_variables])
print(f'Number of Trainable Parameters: {num_trainable_params}')

# Get predictions for the test set
y_true = test_data.labels
y_pred = np.argmax(model.predict(test_data), axis=-1)

# Get classification report for precision, recall, and accuracy
report = classification_report(y_true, y_pred, target_names=test_data.class_indices.keys())

# Print the classification report as a table
print(report)